In [128]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))

shape of x: (569, 30)
shape of y: (569,)


In [129]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
sc = StandardScaler()
x = sc.fit_transform(x)

In [130]:
from torch.utils.data import Dataset, DataLoader
import torch
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length
trainset = dataset(x,y)
#DataLoader
trainloader = DataLoader(trainset,batch_size=60,shuffle=False)

In [131]:
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [132]:
from torch import optim
network = Net(x.shape[1])
optimizer = optim.Adam(network.parameters(), lr=0.001)

In [140]:
from torch.optim.optimizer import Optimizer

class EKFACDistilled(Optimizer):
    def __init__(self, net, eps):
        self.eps = eps
        self.params = []
        self._fwd_handles = []
        self._bwd_handles = []
        self.net = net
        for mod in net.modules():
            mod_class = mod.__class__.__name__
            if mod_class in ['Linear']:
                handle = mod.register_forward_pre_hook(self._save_input)
                self._fwd_handles.append(handle)
                handle = mod.register_full_backward_hook(self._save_grad_output)
                self._bwd_handles.append(handle)
                params = [mod.weight]
                if mod.bias is not None:
                    params.append(mod.bias)
                d = {'params': params, 'mod': mod, 'layer_type': mod_class, 'A': [], 'S': []}
                self.params.append(d)
        super(EKFACDistilled, self).__init__(self.params, {})

    def step(self):
        for group in self.param_groups:
            if len(group['params']) == 2:
                weight, bias = group['params']
            else:
                weight = group['params'][0]
                bias = None
            state = self.state[weight]

            self._compute_kfe(group, state)

            self._precond(weight, bias, group, state)

    def calc_cov(self, calc_act: bool = True):
        for group in self.param_groups:
            if len(group['params']) == 2:
                weight, bias = group['params']
            else:
                weight = group['params'][0]
                bias = None

            state = self.state[weight]

            mod = group['mod']
            x = self.state[group['mod']]['x']
            gy = self.state[group['mod']]['gy']

            # Computation of activation cov matrix for batch
            x = x.data.t()

            # Append column of ones to x if bias is not None
            if mod.bias is not None:
                ones = torch.ones_like(x[:1])
                x = torch.cat([x, ones], dim=0)
            
            if calc_act:
                # Calculate covariance matrix for activations (A_{l-1})
                A = torch.mm(x, x.t()) / float(x.shape[1])
                group['A'].append(A)

            # Computation of psuedograd of layer output cov matrix for batch
            gy = gy.data.t()

            # Calculate covariance matrix for layer outputs (S_{l})
            S = torch.mm(gy, gy.t()) / float(gy.shape[1])

            group['S'].append(S)

    def _compute_kfe(self, group, state):
        mod = group['mod']
        x = self.state[group['mod']]['x']
        gy = self.state[group['mod']]['gy']
        
        # Computation of xxt
        x = x.data.t() # transpose of activations

        # Append column of ones to x if bias is not None
        if mod.bias is not None:
            ones = torch.ones_like(x[:1])
            x = torch.cat([x, ones], dim=0)

        # Calculate covariance matrix for activations (A_{l-1})
        xxt = torch.mm(x, x.t()) / float(x.shape[1])

        # Calculate eigenvalues and eigenvectors of covariance matrix (lambdaA, QA)
        la, state['Qa'] = torch.linalg.eigh(xxt, UPLO='U')

        # Computation of ggt
        gy = gy.data.t()

        # Calculate covariance matrix for layer outputs (S_{l})
        ggt = torch.mm(gy, gy.t()) / float(gy.shape[1])

        # Calculate eigenvalues and eigenvectors of covariance matrix (lambdaS, QS)
        ls, state['Qs'] = torch.linalg.eigh(ggt, UPLO='U')

        # Outer product of the eigenvalue vectors. Of shape (len(s) x len(a))
        state['m2'] = ls.unsqueeze(1) * la.unsqueeze(0)

    def _precond(self, weight, bias, group, state):
        """Applies preconditioning."""
        Qa = state['Qa']
        Qs = state['Qs']
        m2 = state['m2']
        x = self.state[group['mod']]['x']
        gy = self.state[group['mod']]['gy']
        g = weight.grad.data
        s = g.shape
        s_x = x.size()
        s_gy = gy.size()
        bs = x.size(0)

        # Append column of ones to x if bias is not None
        if bias is not None:
            ones = torch.ones_like(x[:,:1])
            x = torch.cat([x, ones], dim=1)
        
        # KFE of activations ??
        x_kfe = torch.mm(x, Qa)

        # KFE of layer outputs ??
        gy_kfe = torch.mm(gy, Qs)

        m2 = torch.mm(gy_kfe.t()**2, x_kfe**2) / bs

        g_kfe = torch.mm(gy_kfe.t(), x_kfe) / bs

        g_nat_kfe = g_kfe / (m2 + self.eps)

        g_nat = torch.mm(g_nat_kfe, Qs.t())

        if bias is not None:
            gb = g_nat[:, -1].contiguous().view(*bias.shape)
            bias.grad.data = gb
            g_nat = g_nat[:, :-1]
        
        g_nat = g_nat.contiguous().view(*s)
        weight.grad.data = g_nat

    def _save_input(self, mod, i):
        """Saves input of layer to compute covariance."""
        self.state[mod]['x'] = i[0]

    def _save_grad_output(self, mod, grad_input, grad_output):
        """Saves grad on output of layer to compute covariance."""
        self.state[mod]['gy'] = grad_output[0] * grad_output[0].size(0)

In [ ]:
import captum._utils.common as common
from captum.influence._core.influence import DataInfluence
from torch.nn import Module
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
from torch import Tensor

class EKFACInfluence(DataInfluence):
    def __init__(
        self,
        module: Module,
        layers: Union[str, List[str]],
        influence_src_dataset: Dataset,
        activation_dir: str,
        model_id: str = "",
        batch_size: int = 1,
        query_batch_size: int = 1,
        **kwargs: Any,
    ) -> None:
        r"""
        Args:
            module (Module): An instance of pytorch model. This model should define all of its
                layers as attributes of the model. The output of the model must be logits for the
                classification task.
            layers (Union[str, List[str]]): A list of layer names for which the influence will
                be computed.
            influence_src_dataset (torch.utils.data.Dataset): Pytorch dataset that is used to create
                a pytorch dataloader to iterate over the dataset. This is the dataset for which we will
                be seeking for influential instances. In most cases this is the training dataset.
            activation_dir (str): Path to the directory where the activation computations will be stored.
            model_id (str): The name/version of the model for which layer activations are being computed.
                Activations will be stored and loaded under the subdirectory with this name if provided.
            batch_size (int): Batch size for the dataloader used to iterate over the influence_src_dataset.
            **kwargs: Any additional arguments that are necessary for specific implementations of the
                'DataInfluence' abstract class.
        """
        self.module = module
        self.layers = [layers] if isinstance(layers, str) else layers
        self.influence_src_dataset = influence_src_dataset
        self.activation_dir = activation_dir
        self.model_id = model_id
        self.batch_size = batch_size
        self.query_batch_size = query_batch_size

        self.influence_src_dataloader = DataLoader(
            self.influence_src_dataset, batch_size=batch_size, shuffle=False
        )
    
    def influence(
            self,
            inputs: Dataset,
            topk: int = 1,
            additional_forward_args: Optional[Any] = None,
            load_src_from_disk: bool = True,
            **kwargs: Any,
        ) -> Dict:

        inputs_batch_size = (
            inputs[0].shape[0] if isinstance(inputs, tuple) else inputs.shape[0]
        )

        influences: Dict[str, Any] = {}
        query_grads: Dict[str, List[Tensor]] = {}
        influence_src_grads: Dict[str, List[Tensor]] = {}

        query_dataloader = DataLoader(
            inputs, batch_size=self.query_batch_size, shuffle=False
        )

        layer_modules = [
            common._get_module_from_name(self.module, layer) for layer in self.layers
        ]

        G_list = self._compute_EKFAC_GNH()

        for i, (queries, targets) in enumerate(query_dataloader):
            criterion = torch.nn.CrossEntropyLoss()
            self.module.zero_grad()
            queries, targets = inputs
            outputs = self.module(queries)
            loss = criterion(outputs, targets.view(-1))
            loss.backward()

            for layer in layer_modules:
                if layer.bias is not None:
                    grad_bias = layer.bias.grad
                    grad_weights = layer.weight.grad
                    grads = torch.cat([grad_weights.view(-1), grad_bias.view(-1)], dim=1)
                else:
                    grads = layer.weight.grad.view(-1)
                for grad in grads:
                    query_grads[layer].append(grad)

        for i, (inputs, targets) in enumerate(self.influence_src_dataloader):
            self.module.zero_grad()
            outputs = self.module(inputs)
            loss = criterion(outputs, targets.view(-1))
            loss.backward()

            for layer in layer_modules:
                if layer.bias is not None:
                    grad_bias = layer.bias.grad
                    grad_weights = layer.weight.grad
                    grads = torch.cat([grad_weights.view(-1), grad_bias.view(-1)], dim=1)
                else:
                    grads = layer.weight.grad.view(-1)
                for grad in grads:
                    influence_src_grads[layer].append(grad)
        
        for layer in layer_modules:
            query_grads[layer] = torch.stack(query_grads[layer])
            influence_src_grads[layer] = torch.stack(influence_src_grads[layer])
            influences[layer] = torch.matmul(influence_src_grads[layer], torch.matmul(G_list[layer], query_grads[layer]).t())

        return influences
            

    def _compute_EKFAC_GNH(self, n_samples: int = 2):
        ekfac = EKFACDistilled(self.module, 1e-5)
        loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
        for i, (input, _) in enumerate(self.influence_src_dataloader):
            outputs = self.module(input)
            output_probs = torch.softmax(outputs, dim=-1)
            distribution = dist.Categorical(output_probs)
            for j in range(n_samples):
                samples = distribution.sample()
                loss = loss_fn(outputs, samples)
                loss.backward()
                ekfac.step()
                self.module.zero_grad()
        
        G_list = []
        # Compute average A and S
        for group in ekfac.param_groups:
            A = torch.stack(group['A']).mean(dim=0)
            S = torch.stack(group['S']).mean(dim=0)
        
            # Compute eigenvalues and eigenvectors of A and S
            la, Qa = torch.linalg.eigh(A, UPLO='U')
            ls, Qs = torch.linalg.eigh(S, UPLO='U')

            # Compute Kronecker product of eigenvalues and eigenvectors
            eigenvec_kron = torch.kron(Qa, Qs)

            eigenval_kron = torch.kron(torch.diag(la),torch.diag(ls))

            # Compute GNH
            G_list.append(torch.matmul(eigenvec_kron, torch.matmul(eigenval_kron, eigenvec_kron.t())))
            
        return G_list

In [134]:
precond = EKFACDistilled(network, eps=0.001)
criterion = torch.nn.BCELoss()

for mod in network.modules():
  mod_class = mod.__class__.__name__
  print(mod_class)
  print("**********************")

Net
**********************
Linear
**********************
Linear
**********************
Linear
**********************


In [135]:
import torch.distributions as dist
loss_fn = nn.CrossEntropyLoss()
n_samples = 2
for i, (inputs, targets) in enumerate(trainloader):
  optimizer.zero_grad
  print(f'Input Shape: {inputs.shape}')
  print(inputs)
  print(f'Target Shape: {targets.shape}')
  outputs = network(inputs)
  print(f'Output Shape: {outputs.shape}')
  output_probs = torch.softmax(outputs, dim=-1)
  distribution = dist.Categorical(output_probs)
  calc_act = True
  for j in range(n_samples):
    samples = distribution.sample()
    loss = loss_fn(outputs, samples)
    loss.backward(retain_graph=True)
    precond.calculate_cov(calc_act=calc_act)
    network.zero_grad()
    calc_act = False
  #loss = criterion(outputs, targets.reshape(-1,1))
  #loss.backward()

Input Shape: torch.Size([60, 30])
tensor([[ 1.0971, -2.0733,  1.2699,  ...,  2.2961,  2.7506,  1.9370],
        [ 1.8298, -0.3536,  1.6860,  ...,  1.0871, -0.2439,  0.2812],
        [ 1.5799,  0.4562,  1.5665,  ...,  1.9550,  1.1523,  0.2014],
        ...,
        [ 0.1655,  0.5353,  0.1475,  ...,  1.0475,  1.2898,  1.4106],
        [-0.3060,  0.0047, -0.3855,  ..., -1.5759, -0.7470, -1.1668],
        [-1.5647, -1.7452, -1.5499,  ..., -1.0722,  0.5165,  0.3499]])
Target Shape: torch.Size([60])
Output Shape: torch.Size([60, 1])
Input Shape: torch.Size([60, 30])
tensor([[-1.1239, -1.0262, -1.1294,  ..., -1.3524,  1.0617, -0.2076],
        [-1.5704,  0.3934, -1.5368,  ..., -1.3221,  0.1153, -0.3783],
        [ 0.0349,  0.6656,  0.1833,  ...,  0.9729, -0.0918,  1.6211],
        ...,
        [ 0.2109, -0.6096,  0.2748,  ...,  1.3414,  1.1070,  1.2499],
        [ 0.4694,  0.8425,  0.5656,  ...,  1.3520,  0.6038,  2.2861],
        [ 1.0857,  0.1676,  0.9157,  ...,  0.0593,  3.2052, -1.2655]])

In [139]:
for group in precond.param_groups:
  print(group['mod'])
  print(f'Activation Cov Matrix len: {len(group["A"])}')
  print(group['A'][0].tolist()[0][:5])
  print(group['A'][1].tolist()[0][:5])
  print(group['A'][2].tolist()[0][:5])
  print(group['A'][3].tolist()[0][:5])
  print(f'Output Cov Matrix len: {len(group["S"])}')
  print(f'Activation Cov Matrix shape: {group["A"][0].shape}')
  print(f'Output Cov Matrix shape: {group["S"][0].shape}')

Linear(in_features=30, out_features=32, bias=True)
Activation Cov Matrix len: 10
[0.8272504210472107, 0.22340252995491028, 0.824838399887085, 0.7830191254615784, 0.0704726055264473]
[1.0926154851913452, 0.41147610545158386, 1.098839521408081, 1.0061488151550293, 0.026723619550466537]
[0.8906545042991638, 0.3817518949508667, 0.8857139348983765, 0.8572977185249329, 0.1744510531425476]
[1.4418561458587646, 0.562223494052887, 1.435240387916565, 1.5619181394577026, 0.19939666986465454]
Output Cov Matrix len: 20
Activation Cov Matrix shape: torch.Size([31, 31])
Output Cov Matrix shape: torch.Size([32, 32])
Linear(in_features=32, out_features=64, bias=True)
Activation Cov Matrix len: 10
[0.09568451344966888, 0.014156293123960495, 0.09352162480354309, 0.012460540048778057, 0.059680547565221786]
[0.17700529098510742, 0.04611698165535927, 0.13976630568504333, 0.0574762187898159, 0.07323087006807327]
[0.18632617592811584, 0.039563439786434174, 0.13833092153072357, 0.024181123822927475, 0.17655216

In [137]:
for group in precond.param_groups:
    A = torch.stack(group['A']).mean(dim=0)
    print(A.tolist()[0][:5])
    S = torch.stack(group['S']).mean(dim=0)

    print(f'Averaged activation matrix: {A.shape}')
    print(f'Averaged output matrix: {S.shape}')

[1.0072457790374756, 0.31007063388824463, 1.0060679912567139, 0.9909204244613647, 0.1981646716594696]
Averaged activation matrix: torch.Size([31, 31])
Averaged output matrix: torch.Size([32, 32])
[0.11650141328573227, 0.031128928065299988, 0.09232742339372635, 0.02067553624510765, 0.08652587234973907]
Averaged activation matrix: torch.Size([33, 33])
Averaged output matrix: torch.Size([64, 64])
[0.02231888845562935, 0.00450670812278986, 0.026402030140161514, 0.0029004632961004972, 0.021502971649169922]
Averaged activation matrix: torch.Size([65, 65])
Averaged output matrix: torch.Size([1, 1])
